In [ ]:
import h5py
import numpy as np
import open3d as o3d
from tqdm import tqdm

## Extract data

In [18]:
# load all .txt files into .h5 format
colored_files = [
    f"../data/raw_data_1/gs100i_2025-07-11-20-26-43colormap_{i}.txt" for i in range(7)
]
all_arrays = []
for f in tqdm(colored_files):
    data = np.loadtxt(f, skiprows=1)
    xyzrgb = data[:, :6]
    all_arrays.append(xyzrgb)
all_arrays = np.vstack(all_arrays)
processed_file = "../data/processed_data/colored.h5"
with h5py.File(processed_file, "w") as h5f:
    h5f.create_dataset("coordinates", data=all_arrays[:, :3])
    h5f.create_dataset("colors", data=all_arrays[:, 3:])

100%|██████████| 7/7 [00:18<00:00,  2.63s/it]


In [31]:
# read from .h5
with h5py.File(processed_file, "r") as h5f:
    for i, row in enumerate(zip(h5f["coordinates"], h5f["colors"])):
        print(row)
        if i >= 5:
            break

(array([ -9.34500027, -34.02700043,  15.2840004 ]), array([ 90., 166., 254.]))
(array([ -9.38700008, -32.91400146,  14.79399967]), array([ 91., 162., 252.]))
(array([ -8.64599991, -28.54100037,  12.91600037]), array([151., 174., 139.]))
(array([ -8.0010004 , -25.81699944,  11.68900013]), array([255., 249., 249.]))
(array([ -7.94999981, -25.0720005 ,  11.38199997]), array([154., 174., 198.]))
(array([ -7.90299988, -24.6439991 ,  11.20199966]), array([72., 90., 43.]))


## Visualization

In [47]:
with h5py.File(processed_file, "r") as f:
    coords = f["coordinates"][:]
    colors = f["colors"][:]

In [ ]:
# render full scene
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(coords)
pcd.colors = o3d.utility.Vector3dVector(colors / 255)
o3d.visualization.draw_geometries([pcd])

In [66]:
coords[(coords[:, 2] > 0) & (coords[:, 2] < 10)].shape

(15207399, 3)

In [80]:
# max and min height
coords[:, 2].min(), coords[:, 2].max()

(np.float64(-3.94799995), np.float64(46.77600098))

In [ ]:
# render only those dots that are >-2 and <0.3 (used for extracting stumps)
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(
    coords[(coords[:, 2] > -2) & (coords[:, 2] < 0.3)]
)
pcd.colors = o3d.utility.Vector3dVector(
    colors[(coords[:, 2] > -2) & (coords[:, 2] < 0.3)] / 255
)
o3d.visualization.draw_geometries([pcd])